In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import statistics as s

In [2]:
from functions import log

In [3]:
fileList_list = ['files/file_list_0.txt', 'files/file_list_1.txt']

In [4]:
all_df = []
for file in fileList_list:
    df = pd.read_csv(file, delimiter= '\t')
    all_df.append(df)

df = pd.concat(all_df)
    

In [5]:
df = df[df["Article Citation"].str.startswith("PLoS")]
df

,Article File,Article Citation,AccessionID,LastUpdated (YYYY-MM-DD HH:MM:SS),PMID,License,Retracted
0,PMC000xxxxxx/PMC176545.xml,PLoS Biol. 2003 Oct 18; 1(1):e5,PMC176545,2022-01-13 00:01:34,12929205,CC BY,no
1,PMC000xxxxxx/PMC176546.xml,PLoS Biol. 2003 Oct 18; 1(1):e6,PMC176546,2022-01-13 00:01:35,12929206,CC BY,no
2,PMC000xxxxxx/PMC176547.xml,PLoS Biol. 2003 Oct 18; 1(1):e7,PMC176547,2022-01-13 00:01:35,0,CC BY,no
3,PMC000xxxxxx/PMC176548.xml,PLoS Biol. 2003 Oct 18; 1(1):e11,PMC176548,2022-01-13 00:01:34,0,CC BY,no
4,PMC000xxxxxx/PMC193604.xml,PLoS Biol. 2003 Oct 15; 1(1):e13,PMC193604,2022-01-13 00:01:34,12975658,CC BY,no
5,PMC000xxxxxx/PMC193605.xml,PLoS Biol. 2003 Oct 15; 1(1):e19,PMC193605,2022-01-13 00:01:34,12975657,CC BY,no
6,PMC000xxxxxx/PMC193606.xml,PLoS Biol. 2003 Oct 15; 1(1):e23,PMC193606,2022-01-13 00:01:34,0,CC BY,no
7,PMC000xxxxxx/PMC193607.xml,PLoS Biol. 2003 Oct 15; 1(1):e31,PMC193607,2022-01-13 00:01:34,0,CC BY,no
9,PMC000xxxxxx/PMC212687.xml,PLoS Biol. 2003 Oct 13; 1(1):e1,PMC212687,2022-01-13 00:01:34,14551903,CC BY,no
10,PMC000xxxxxx/PMC212688.xml,PLoS Biol. 2003 Oct 13; 1(1):e4,PMC212688,2022-01-13 00:01:34,0,CC BY,no


In [6]:
# initialize empty lists
titles = []
abstracts = []
refs = []

In [7]:
def main(article_file):
    tree = ET.parse('./files/'+article_file) # parse file
    root = tree.getroot() # get root
    
    # get title
    title_element = root.findall('front')[0].findall('article-meta')[0].findall('title-group')[0].findall('article-title')[0]
    title = "".join(title_element.itertext())
    title = title.replace('\n','')
    
    # get abstract
    try:
        abstract_element = root.findall('front')[0].findall('article-meta')[0].findall('abstract')[0]
        abstract = "".join(abstract_element.itertext())
        abstract = abstract.replace('\n','')
    except: # no abstract, take body?
        abstract_element = root.findall('body')[0]
        abstract = "".join(abstract_element.itertext())
        abstract = abstract.replace('\n','')
    
    # get references
    pmid_list = []
    try:
        reference_element = root.findall('back')[0].findall('ref-list')[0].findall('ref')
        counter = 0
        for ref in reference_element:
            try:
                pmid = "".join(ref.findall('element-citation')[0].findall('pub-id')[0].itertext())
                pmid_list.append(pmid)
            except: # no pmid
                pass
    except:
        pmid_list = "NA"
        
    titles.append(title)
    abstracts.append(abstract)
    refs.append(pmid_list)
    
    log(f'Ran select_files.ipynb succesfully')
        

In [8]:
for i in df["Article File"]:
    main(i)

In [9]:
df["title"] = titles
df["abstract"] = abstracts
df["references"] = refs

In [11]:
lens = df['abstract'].str.split().apply(len)
print(max(lens))
print(min(lens))
print(s.mean(lens))
print(s.stdev(lens))

6911
1
308.5137834598482
404.05820703843295


In [12]:
df.to_csv(r'files/select_files.txt', sep='\t', index = False)